# Overhead Signs Work Order Creation
--------------------------------------------------
The purpose of this script is to create work orders of overhead street name signs. The information for work orders are created from a feature layer of operational maintenance areas intersecting signalized intersections.


## Imports

In [20]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayer,GeoAccessor, GeoSeriesAccessor
import pandas as pd

Access the COA login

In [ ]:
gis = GIS("https://austin.maps.arcgis.com/home/index.html", client_id='CrnxPfTcm7Y7ZGl7')

Retrieve locations of signalized intersections intersecting operational maintenance areas to publish into a folder and excel worksheet.

In [22]:
url = r"https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/{}/FeatureServer/0"
signals = pd.DataFrame.spatial.from_layer(FeatureLayer(url.format("TRANSPORTATION_signals2")))
oma = pd.DataFrame.spatial.from_layer(FeatureLayer(url.format("LOCATION_grids_200"))) #OMA layer will be on AGOL
display(oma.head())
#overhead = oma.overlay(signals)

#overhead = signals.intersect(oma,1)

,OBJECTID,TILE_NAME,SHAPE
0,1,R52,"{""rings"": [[[3161341.49986032, 10201803.000153..."
1,2,M52,"{""rings"": [[[3135138.25010157, 10201165.000117..."
2,3,J52,"{""rings"": [[[3115485.25003757, 10200699.000065..."
3,4,WS54,"{""rings"": [[[3010493.25004824, 10206922.000086..."
4,5,V53,"{""rings"": [[[3187435.24983807, 10206722.000158..."


In [2]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from PIL import Image
from resizeimage import resizeimage
from collections import OrderedDict
import csv
import pandas as pd
import datetime
import sys
import os
import win32com.client
from knackpy import Knack

In [2]:
# This will create the list of values for each intersection based on signal ID
filePath = r"G:\ATD\ATD_GIS\Signs\123_Signs_Maintenance_Plan\Operational_Maintenance_Areas_Sign_Maintenance_Plan"
author = "Susanne Gov"

df = pd.read_excel(filePath + r"\OverheadSigns_FY2019.xlsx", sheet_name='Signs')  # sheetname is optional
df.to_csv(filePath + r'\OverheadSigns_FY2019_SIGNS.csv', index=False)

work_orders = pd.read_excel(filePath + r"\OverheadSigns_FY2019.xlsx", sheet_name='StreetView Link')  # sheetname is optional
work_orders.to_csv(filePath + r'\OverheadSigns_FY2019.csv', index=False)          
signalID = []
index = 1
with open (filePath + r'\OverheadSigns_FY2019.csv','rU') as csvfile:
    orderRow = csv.reader(csvfile, delimiter=',')
    for row in orderRow:
        values = OrderedDict()
        if row[0] != "SIGNAL_ID":
            values["Created Date"] = row[15]
            values["Created By"] = author
            values["Work Order ID"] = row[14]
            values["Signal ID"] = row[0]
            values["Intersection ID"] = row[1]
            values["Primary Street"] = row[2]
            values["Cross Street"] = row[3]
            values["Link (N)"] = row[10]
            values["Link (S)"] = row[11]
            values["Link (E)"] = row[12]
            values["Link (W)"] = row[13]
            signalID.append(values)
            index +=1
    index = 1
    csvfile.close()

In [3]:
signs = []

with open (filePath + r'\OverheadSigns_FY2019_SIGNS.csv','rU') as csvfile:
    csv = csv.reader(csvfile, delimiter=',')
    for row in csv:
        sign = OrderedDict()
        sign["Signal ID"] = row[0]
        sign["Direction"] = row[1]
        sign["Sign Type"] = row[2] 
        sign["Street Sign"] = row[3]
        sign["Bottom Text (Optional)"] = row[6]
        sign["Install/Remove"] = row[7]
        signs.append(sign)

In [4]:
# Take an existing image of the streetview imagery and will resize the image to fit the work order template
def makeImg (sigId, nSlide):
    imgname = sigId + "_" + direction
    imagery = imagePath + "/" + imgname + ".png"
    try:
        with open(imagery, 'r+b') as f:
            with Image.open(f) as image:
                cover = resizeimage.resize_contain(image, [560, 280])
                cover.save(imagePath + "Cropped/" + imgname + ".png", image.format)
        pic = nSlide.add_picture(imagePath + "Cropped/" + imgname + ".png",Inches(0.25),Inches(1.1))
        line = pic.line
        line.color.rgb = RGBColor(0,0,0)
        line.width = Inches(0.05)
    except IOError:
        return None        

In [5]:
# Create a pretty header
def makeHead(nSlide):
    title = nSlide.add_textbox(Inches(0.25),Inches(0.4),Inches(7.5),Inches(0.75))
    tf = title.text_frame
    tf.clear()
    p = tf.paragraphs[0]
    run = p.add_run()
    run.text = "Austin Transportation Department Signs Work Orders"
    font = run.font
    font.name = 'Arial'
    font.size = Pt(20)
    font.bold = True
    seal = "G:/ATD/ATD_GIS/03_RESOURCES/Seals_Logos/cityseal_60_x_60.jpg"
    pic = nSlide.add_picture(seal,Inches(8.5),Inches(0.25))

In [6]:
# Create information box for work orders
def makeInfo(sigID, nSlide, direction):
    info = nSlide.add_textbox(Inches(8.25),Inches(1.15),Inches(1.5),Inches(4.5))
    infotext = info.text_frame.paragraphs[0]
    info.text_frame.word_wrap = True
    for (key,val) in sigID.iteritems():
        if "Link" not in key or direction in key:
            run1 = infotext.add_run()
            run1.text = key + "\n"
            run1.font.size = Pt(12)
            run1.font.bold = True
            run1.font.underline = True
            run2 = infotext.add_run();
            if "Link" in key and "N/A" not in val:
                run2.hyperlink.address = val
            run2.text =  val + "\n"
            run2.font.size = Pt(12)
    run3 = infotext.add_run()
    run3.text = "Cardinal Direction: "
    run3.font.bold = True
    run3.font.underline = True  
    run3.font.size = Pt(12)
    c = infotext.add_run()
    if direction is "N":
        c.text = "North"
    elif (direction is "S"):
        c.text = "South"
    elif (direction is "E"):
        c.text = "East"
    elif (direction is "W"):
        c.text = "West"
    c.font.size = Pt(12)   

In [7]:
# For changing table font sizes
def iter_cells(table):
    for row in table.rows:
        for cell in row.cells:
            yield cell
# Create list of Sign Overheads
def makeSignTable(sigID,nSlide,direction,signs):
    # row/column/left/top/width/height
    table = nSlide.add_table(6,4,Inches(0.25),Inches(5.2),170,1000).table
    table.cell(0,0).text = "Sign Type"
    table.cell(0,1).text = "Street Sign"
    table.cell(0,2).text = "Bottom Text"
    table.cell(0,3).text = "Install/Remove"
    
    table.columns[0].width = Inches(5)
    table.columns[1].width = Inches(1.75)
    table.columns[2].width = Inches(1.5)
    table.columns[3].width = Inches(1.3)
    
    index = 1
    for x in signs:
        if x["Signal ID"] == sigID and x["Direction"] == direction:
            table.cell(index,0).text = x["Sign Type"]
            table.cell(index,1).text = x["Street Sign"]
            table.cell(index,2).text = x["Bottom Text (Optional)"]
            table.cell(index,3).text = x["Install/Remove"]
            index +=1
    for cell in iter_cells(table):
        for paragraph in cell.text_frame.paragraphs:
            for run in paragraph.runs:
                run.font.size = Pt(12)

In [8]:
# Converts Powerpoint file to PDF file
def convertPDF(path,name):
    in_file = path + "/" + name
    out_file = path + "PDF\\" + name[:-5] + ".pdf"
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")
    pdf = powerpoint.Presentations.Open(in_file,WithWindow=False)
    pdf.SaveCopyAs(out_file,32)
    pdf.Close()
    powerpoint.Quit()

In [9]:
dirs = ["N","S","E","W"]
path = "G:\\ATD\\ATD_GIS\\Signs\\123_Signs_Maintenance_Plan\\Operational_Maintenance_Areas_Sign_Maintenance_Plan\\WorkOrderSignsOverheadFY2019"
imagePath = "G:/ATD/ATD_GIS/Signs/123_Signs_Maintenance_Plan/Operational_Maintenance_Areas_Sign_Maintenance_Plan/SignsFY2019Imagery"

for valueDictionary in signalID:
    workOrder = Presentation()
    dirindex = 0
    for direction in dirs:
        if "N/A" not in valueDictionary["Link ("+ direction + ")"]:
            blank_slide_layout = workOrder.slide_layouts[6]
            slide = workOrder.slides.add_slide(blank_slide_layout)
            nSlide = slide.shapes
            makeImg(valueDictionary["Signal ID"], nSlide)
            makeSignTable(valueDictionary["Signal ID"],nSlide,direction,signs)
            makeHead(nSlide)
            makeInfo(valueDictionary, nSlide, direction)
        dirindex += 1
    name = "ATDSignsWorkOrders_" + valueDictionary["Signal ID"] + ".pptx"
    workOrder.save(str(path) + "/" + name)
    convertPDF(path,name)